In [114]:
import time
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


# Representação do problema

In [115]:
pessoas = [('Lisbon', 'LIS'), ('Madrid', 'MAD'), ('Paris', 'CDG'), ('Dublin', 'DUB'), ('Brussels', 'BRU'), ('Lodon', 'LHR')]
destino = 'FCO'
dominio_voos = [(0, 9)] * (len(pessoas) * 2)
voos = {}
for linha in open('flights.txt'):
    origin, destino, partida, chegada, preco = linha.split(',')
    voos.setdefault((origin, destino), []).append([partida, chegada, int(preco.strip())])


In [116]:
def imprime_calendario(calendario):
    voo_id = -1
    preco_total = 0
    for it_calendario in range(len(calendario) // 2):
        nome = pessoas[it_calendario][0]
        origem = pessoas[it_calendario][1]
        voo_id += 1
        voo_ida = voos[(origem, destino)][calendario[voo_id]]
        preco_total += voo_ida[2]
        voo_id += 1
        voo_volta = voos[(destino, origem)][calendario[voo_id]]
        preco_total += voo_volta[2]
        print('%10s%10s %5s-%5s U$%3s %5s-%5s U$%3s' % (nome, origem, voo_ida[0], voo_ida[1], voo_ida[2],
                                                        voo_volta[0], voo_volta[1], voo_volta[2]))
    print('Preço total: ', preco_total)

In [117]:
def get_minutos(hora):
    t = time.strptime(hora, '%H:%M')
    minutos = t[3] * 60 + t[4]
    return minutos

# Função de avaliação (Fitness function)

In [124]:
%%time
def funcao_avaliacao_voos(calendario):
    preco_total = 0
    ultima_chegada = 0
    primeira_partida = 1439

    voo_id = -1
    for ite_calendario in range(len(calendario) // 2):
        origem = pessoas[ite_calendario][1]
        voo_id += 1
        voo_ida = voos[(origem, destino)][calendario[voo_id]]
        voo_id += 1
        voo_volta = voos[(destino, origem)][calendario[voo_id]]

        preco_total += voo_ida[2]
        preco_total += voo_volta[2]

        if ultima_chegada < get_minutos(voo_ida[1]):
            ultima_chegada = get_minutos(voo_ida[1])
        if primeira_partida > get_minutos(voo_volta[0]):
            primeira_partida = get_minutos(voo_volta[0])

    espera_total = 0
    voo_id = -1

    for ite_calendario in range(len(calendario) // 2):
        origem = pessoas[ite_calendario][1]
        voo_id += 1
        voo_ida = voos[(origem, destino)][calendario[voo_id]]
        voo_id += 1
        voo_volta = voos[(destino, origem)][calendario[voo_id]]

        espera_total += ultima_chegada - get_minutos(voo_ida[1])
        espera_total += get_minutos(voo_volta[0]) - primeira_partida

    return espera_total + preco_total



Error compiling Cython file:
------------------------------------------------------------
...
    ultima_chegada = 0
    primeira_partida = 1439

    voo_id = -1
    for ite_calendario in range(len(calendario) // 2):
        origem = pessoas[ite_calendario][1]
                ^
------------------------------------------------------------

/home/victor/.cache/ipython/cython/_cython_magic_bc49c76ccf2a727860ed29f07b5c0485.pyx:8:17: undeclared name not builtin: pessoas

Error compiling Cython file:
------------------------------------------------------------
...

    voo_id = -1
    for ite_calendario in range(len(calendario) // 2):
        origem = pessoas[ite_calendario][1]
        voo_id += 1
        voo_ida = voos[(origem, destino)][calendario[voo_id]]
                 ^
------------------------------------------------------------

/home/victor/.cache/ipython/cython/_cython_magic_bc49c76ccf2a727860ed29f07b5c0485.pyx:10:18: undeclared name not builtin: voos

Error compiling Cython file

In [119]:
import random

def mutacao(dominio, passo, calendario, probabilidade):
    gene = random.randint(0, len(dominio) - 1)
    mutante = calendario
    if random.random() < probabilidade:
        if calendario[gene] != dominio[gene][0]:
            mutante = calendario[0:gene] + [calendario[gene] - passo] + calendario[gene + 1:]
        elif calendario[gene] != dominio[gene][1]:
            mutante = calendario[0:gene] + [calendario[gene] + passo] + calendario[gene + 1:]
    return mutante

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 5.72 µs


# Crossover

In [120]:
def crossover(dominio, individuo1, individuo2):
    gene = random.randint(1, len(dominio) - 2)
    return individuo1[0:gene] + individuo2[gene:]

In [121]:
def algoritmo_genetico(dominio, funcao_avaliacao, tamanho_populacao = 100, passo = 1, elitismo = 0.2, numero_geracoes = 500, probabilidade_mutacao = 0.05):
    populacao = []
    for _ in range(tamanho_populacao):
        individuo = [random.randint(dominio[i][0], dominio[i][1]) for i in range(len(dominio))]
        populacao.append(individuo)
    numero_elitismo = int(elitismo * tamanho_populacao)
    custos = []
    for _ in range(numero_geracoes):
        custos = [(funcao_avaliacao(individuo), individuo) for individuo in populacao]
        custos.sort()
        individuos_ordenados = [individuo for (custo, individuo) in custos]
        populacao = individuos_ordenados[0:numero_elitismo]

        while len(populacao) < tamanho_populacao:
            i1 = random.randint(0, numero_elitismo)
            i2 = random.randint(0, numero_elitismo)

            novo_individuo = crossover(dominio, individuos_ordenados[i1], individuos_ordenados[i2])

            mutacao_novos_individuos = mutacao(dominio, passo, novo_individuo, probabilidade_mutacao)
            populacao.append(mutacao_novos_individuos)
    return custos[0][1]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [122]:
%%time

menor_solucao = [5000,[]]
for _ in range(0,100):
    solucao = algoritmo_genetico(dominio_voos, funcao_avaliacao_voos, numero_geracoes = 100, tamanho_populacao=100,
                                 elitismo = 0.2, probabilidade_mutacao = 0.05)
    if funcao_avaliacao_voos(solucao) < menor_solucao[0]:
        menor_solucao[0] = funcao_avaliacao_voos(solucao)
        menor_solucao[1] = solucao

print(menor_solucao[0])
imprime_calendario(menor_solucao[1])

2354
    Lisbon       LIS 12:18-14:56 U$172  8:04-10:59 U$136
    Madrid       MAD 12:44-14:17 U$134  7:50-10:08 U$164
     Paris       CDG 11:28-14:40 U$248  8:23-11:07 U$143
    Dublin       DUB 12:34-15:02 U$109  8:23-10:28 U$149
  Brussels       BRU 10:30-14:57 U$290  9:49-13:51 U$229
     Lodon       LHR 12:08-14:59 U$149  8:19-11:16 U$122
Preço total:  2045
CPU times: user 1min 51s, sys: 0 ns, total: 1min 51s
Wall time: 1min 51s
